Evaluate the initial performance of the MedSAM off-the-shelf (code copied from example notebook) (https://colab.research.google.com/drive/19WNtRMbpsxeqimBlmJwtd1dzpaIvK2FZ?usp=sharing#scrollTo=MBWWGgitt9D2). 

Please download the model medsam_vit_b.pth to your directory ahead of running this notebook. https://drive.google.com/drive/folders/1ETWmi4AiniJeWOt6HAsYgTjYv_fkgzoN

Clone both of the following repository as well. Some files from axondeepseg are changed slightly so that the import statements work outside the packages. Please reference the repo for these changes.

In [ ]:
import MedSAM.train_one_gpu
!git clone https://github.com/axondeepseg/axondeepseg.git
!git clone https://github.com/bowang-lab/MedSAM

In [187]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import torch
from segment_anything import sam_model_registry
from skimage import io, transform
import torch.nn.functional as F
from axondeepseg.AxonDeepSeg.testing.segmentation_scoring import Metrics_calculator

In [188]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251 / 255, 252 / 255, 30 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)


def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(plt.Rectangle((x0, y0), w, h, edgecolor='blue', facecolor=(0, 0, 0, 0), lw=2))


@torch.no_grad()
def medsam_inference(medsam_model, img_embed, box_1024, H, W):
    box_torch = torch.as_tensor(box_1024, dtype=torch.float, device=img_embed.device)
    if len(box_torch.shape) == 2:
        box_torch = box_torch[:, None, :]  # (B, 1, 4)

    sparse_embeddings, dense_embeddings = medsam_model.prompt_encoder(
        points=None,
        boxes=box_torch,
        masks=None,
    )
    low_res_logits, _ = medsam_model.mask_decoder(
        image_embeddings=img_embed,  # (B, 256, 64, 64)
        image_pe=medsam_model.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
        sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
        dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
        multimask_output=False,
    )

    low_res_pred = torch.sigmoid(low_res_logits)  # (1, 1, 256, 256)

    low_res_pred = F.interpolate(
        low_res_pred,
        size=(H, W),
        mode="bilinear",
        align_corners=False,
    )  # (1, 1, gt.shape)
    low_res_pred = low_res_pred.squeeze().cpu().numpy()  # (256, 256)
    medsam_seg = (low_res_pred > 0.5).astype(np.uint8)
    return medsam_seg


MedSAM_CKPT_PATH = "medsam_vit_b.pth"
device = "cuda:0"
medsam_model = sam_model_registry['vit_b'](checkpoint=MedSAM_CKPT_PATH)
medsam_model = medsam_model.to(device)
medsam_model.eval()

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


We need to also generate the bounding boxes from ground truth masks

In [ ]:


def to_boxes(mask):
    mask = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    for i in range(mask.shape[1]):
        for j in range(mask.shape[0]):
            if mask[i, j] != 0:
                mask[i, j] = 1
    print(mask)


to_boxes("datasets/zenodo/labels/7_EM1_3_10D_N_P_0002.png")

In [ ]:
def eval_image(img, ground_truth, plot=False):
    img_np = cv2.imread(img)
    if len(img_np.shape) == 2:
        img_3c = np.repeat(img_np[:, :, None], 3, axis=-1)
    else:
        img_3c = img_np
    H, W, _ = img_3c.shape
    img_1024 = transform.resize(img_3c, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)
    img_1024 = (img_1024 - img_1024.min()) / np.clip(
        img_1024.max() - img_1024.min(), a_min=1e-8, a_max=None
    )  # normalize to [0, 1], (H, W, 3)
    # convert the shape to (3, H, W)
    img_1024_tensor = torch.tensor(img_1024).float().permute(2, 0, 1).unsqueeze(0).to(device)

    box_np = np.array([[0, 0, 200, 100]])  # edit: need to make it detect the entire image
    # transfer box_np t0 1024x1024 scale 
    box_1024 = box_np / np.array([W, H, W, H]) * 1024
    with torch.no_grad():
        image_embedding = medsam_model.image_encoder(img_1024_tensor)  # (1, 256, 64, 64)

    medsam_seg = medsam_inference(medsam_model, image_embedding, box_1024, H, W)

    if plot:
        fig, ax = plt.subplots(1, 2, figsize=(10, 5))
        ax[0].imshow(img_3c)
        show_box(box_np[0], ax[0])
        ax[0].set_title("Input Image and Bounding Box")
        ax[1].imshow(img_3c)
        show_mask(medsam_seg, ax[1])
        show_box(box_np[0], ax[1])
        ax[1].set_title("MedSAM Segmentation")
        plt.show()

    # calculate dice score
    ground_truth_mask_axon = cv2.imread(ground_truth, cv2.IMREAD_GRAYSCALE) // 255
    m_axon = Metrics_calculator(medsam_seg, ground_truth_mask_axon)
    return m_axon.pw_dice()

In [ ]:
# plot the performance of this image, for instance
print(
    f"The dice score is {eval_image('datasets/zenodo/images/7_EM1_3_10D_N_P_0002.png', 'datasets/zenodo/labels/7_EM1_3_10D_N_P_0002.png', plot=True)}"
)

Calculate the mean dice score across all the different datasets


In [ ]:
def evaluate_dataset(dataset):
    images = Path(dataset).joinpath("images").glob("*")
    num_images = 0
    tot_dice = 0
    for image in images:
        label = Path(dataset).joinpath("labels", image.name)
        tot_dice += eval_image(str(image), str(label))
        num_images += 1
    print(f"The mean dice score of {Path(dataset).name} is {tot_dice / num_images:.5f}")
    return tot_dice / num_images, num_images


mean_dice_zenodo, num_zenodo = evaluate_dataset("datasets/zenodo")
mean_dice_osf, num_osf = evaluate_dataset("datasets/osfstorage-archive")
mean_dice_ads, num_ads = evaluate_dataset("datasets/data_axondeepseg_sem")
print(
    f"The mean dice score is {(mean_dice_zenodo * num_zenodo + mean_dice_osf * num_osf + mean_dice_ads * num_ads) / (num_zenodo + num_ads + num_osf):.5f}"
)


Now, we can start to finetune the model with these images. We will need to first augment the datasets and preprocess them to required format of MedSam's train_one_gpu script.

In [193]:
from aug import *


def dataset_to_npy(dataset):
    images = Path(dataset).joinpath("images").glob("*")

    def save_npy(image, image_np, label_myelin, label_axons, augmentation=None):
        img_npy_path_m = Path(dataset).parent.joinpath("npy", "myelin", "imgs")
        gt_npy_path_m = Path(dataset).parent.joinpath("npy", "myelin", "gts")
        img_npy_path_a = Path(dataset).parent.joinpath("npy", "axon", "imgs")
        gt_npy_path_a = Path(dataset).parent.joinpath("npy", "axon", "gts")
        
        if np.random.uniform() < 0.1: # 10% test dataset
            img_npy_path_m = Path(dataset).parent.joinpath("test", "myelin", "imgs")
            gt_npy_path_m = Path(dataset).parent.joinpath("test", "myelin", "gts")
            img_npy_path_a = Path(dataset).parent.joinpath("test", "axon", "imgs")
            gt_npy_path_a = Path(dataset).parent.joinpath("test", "axon", "gts")
        
        with open(str(img_npy_path_m.joinpath(
            f"{Path(dataset).name}-{image.name}{'-'+augmentation if augmentation is not None else ''}.npy"
        )), 'wb') as f:
            np.save(f, image_np)
        with open(str(img_npy_path_a.joinpath(
            f"{Path(dataset).name}-{image.name}{'-'+augmentation if augmentation is not None else ''}.npy"
        )), 'wb') as f:
            np.save(f, image_np)
        with open(str(gt_npy_path_m.joinpath(
            f"{Path(dataset).name}-{image.name}{'-'+augmentation if augmentation is not None else ''}.npy"
        )), 'wb') as f:
            np.save(f, label_myelin)
        with open(str(gt_npy_path_a.joinpath(
            f"{Path(dataset).name}-{image.name}{'-'+augmentation if augmentation is not None else ''}.npy"
        )), 'wb') as f:
            np.save(f, label_axons)

    augmentations = [
        random_shift_2d,
        random_rotation_2d,
        random_rescale_2d,
        random_flip_2d,
        random_blur_2d
    ]

    for image in images:
        label = Path(dataset).joinpath("labels", image.name)
        label = cv2.imread(str(label), cv2.IMREAD_GRAYSCALE)
        label = transform.resize(label, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)
        label_axons = np.zeros(label.shape)
        label_myelin = np.zeros(label.shape)

        image_np = cv2.imread(str(image), cv2.IMREAD_GRAYSCALE)
        image_np = transform.resize(image_np, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(
            np.uint8)
        image_np = (image_np - image_np.min()) / np.clip(
            image_np.max() - image_np.min(), a_min=1e-8, a_max=None
        )
        for i in range(label.shape[1]):
            for j in range(label.shape[0]):
                if label[i, j] == 255:
                    label_axons[i, j] = 1
                if label[i, j] == 127:
                    label_myelin[i, j] = 1

        save_npy(image, np.dstack([image_np for _ in range(3)]), label_myelin, label_axons)

        for augmentation in augmentations:
            augmented_image = augmentation(image_np)
            augmented_axon = augmentation(label_axons)
            augmented_myelin = augmentation(label_myelin)

            save_npy(image, np.dstack([augmented_image for _ in range(3)]), augmented_axon, augmented_myelin, augmentation.__name__)

dataset_to_npy("datasets/data_axondeepseg_sem")
dataset_to_npy("datasets/zenodo")
dataset_to_npy("datasets/osfstorage-archive")


Training. You need to either remove the sanity check from the training file or install the CT dataset
for it to function.

In [189]:
!python3 MedSAM/train_one_gpu.py --tr_npy_path datasets/npy/axon -checkpoint medsam_vit_b.pth -num_epochs 1

/home/shing6444/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
Number of total parameters:  93735472
Number of trainable parameters:  93729252
Number of image encoder and mask decoder parameters:  93729252
number of images: 127
Number of training samples:  127
  0%|                                                    | 0/64 [00:00<?, ?it/s]
Traceback (most recent call last):
  File "/home/shing6444/PycharmProjects/Segment-Any-Axon/MedSAM/train_one_gpu.py", line 350, in <module>
    main()
  File "/home/shing6444/PycharmProjects/Segment-Any-Axon/MedSAM/train_one_gpu.py", line 291, in main
    for step, (image, gt2D, boxes, _) in enumerate(tqdm(train_dataloader)):
  File "/home/s